In [ ]:
# importing various required libraries and modules

import os  # for dealing with files and directories
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for visualization and also for reading & displaying images
from keras.preprocessing.image import ImageDataGenerator # for Data Augmentation
from keras.applications.resnet50 import ResNet50 #Model to be trained
import keras

In [ ]:
train_df = pd.read_csv('../input/input-csv-files/train_df.csv')
test_df = pd.read_csv('../input/input-csv-files/test.csv')
valid_df = pd.read_csv('../input/input-csv-files/valid_df.csv')
link = '../input/whatsappimagedataset'
class_dic = {0:'paper',1:'poster',2:'meme',3:'screenshot',4:'family',5:'others'}
train_df['label'].replace(class_dic,inplace=True)
test_df['label'].replace(class_dic,inplace=True)
valid_df['label'].replace(class_dic,inplace=True)

In [ ]:
train_data_gen = ImageDataGenerator(
    rescale=1./255, 
    zoom_range=0.1,
    rotation_range=10,
    brightness_range=[0.5,1.2],
    width_shift_range=0.1,
    height_shift_range=0.1
)
train_data = train_data_gen.flow_from_dataframe(
    train_df,
    link,
    x_col='filename',
    y_col='label',
    class_mode='categorical',
    target_size=(224,224),
    batch_size=32
)

In [ ]:
valid_data_gen = ImageDataGenerator(rescale=1./255)
valid_data = valid_data_gen.flow_from_dataframe(
    valid_df,
    link,
    x_col='filename',
    y_col='label',
    class_mode='categorical',
    target_size=(224,224),
    batch_size=32
)

In [ ]:
test_data_gen = ImageDataGenerator(rescale=1./255)
test_data = test_data_gen.flow_from_dataframe(
    test_df,
    link,
    x_col='filename',
    y_col='label',
    class_mode='categorical',
    target_size=(224,224),
    batch_size=32
)

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
layer1 = keras.layers.Dense(300,activation='elu',kernel_initializer='he_normal')(avg)
drop1 = keras.layers.Dropout(rate=0.2)(layer1)
layer2 = keras.layers.Dense(300,activation='elu',kernel_initializer='he_normal')(drop1)
drop2 = keras.layers.Dropout(rate=0.2)(layer2)
output = keras.layers.Dense(6, activation='softmax')(drop2) # because the number of classes are 6
model = keras.Model(inputs=base_model.input, outputs=output)

In [ ]:
# freezing all the layers of the model except the top one
base_model.trainable = False

### Training the model

In [ ]:
model.summary()

In [ ]:
set_trainable = False
for layer in model.layers:
    if layer.name in ['conv5_block1_1_conv']:
        set_trainable = True
    layer.trainable = set_trainable

In [ ]:
optimizer=keras.optimizers.Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
checkpoint_cb=keras.callbacks.ModelCheckpoint('project_cnn.h5',save_best_only=True)
early_stopping_cb=keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)
learning_rate_cb=keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=3,verbose=1)

In [ ]:
history= model.fit(
    train_data, 
    epochs=100,
    validation_data=valid_data,
    callbacks=[learning_rate_cb,early_stopping_cb,checkpoint_cb])